In [1]:
import os
import pandas as pd

In [22]:
if not os.path.exists('./jw_utils'):
    !git clone https://github.com/JonWinkelman/jw_utils.git
from jw_utils import parse_gff as pgf

### Get appropriate files and fill in paths  
**Files needed:**  
    1. raw compressed fastq files (.fastq.gz)  
    3. GTF or GFF annotation file  
    4. Genomic sequence in fasta format  
    5.  samplesheet.csv  
    
**Paths to fill in:**  
    1. path to AWS project dir
    2. genome filename
    3. gff or gtf filename
    
**Structure of AWS project dir**  
Folders:
1. raw/  
     - fastq files  
2. references/  
     - annotation files  
     - genome sequence  
3. assets/  
     - samplesheet.csv  
4. results/  

## Fill in these paths!!
**This assumes that the AWS 'assets' and 'references' directories have been populated   
with the references/<genome>.fasta, references/<annotation>.gff and assets/samplesheet.csv**

In [23]:
aws_project_dir = "s3://mukherjee-lab/dimA_RNA_seq_2025/"
genome_filename = 'GCA_000014625.1_ASM1462v1_genomic.fna'
annotation_filename = 'genomic.gff'

#### Create aws file paths:

In [26]:
annotation_fp = os.path.join(aws_project_dir, 'references/', annotation_filename)
genome_fp = os.path.join(aws_project_dir, 'references/', genome_filename)
samplesheet_fp = os.path.join(aws_project_dir,'assets/samplesheet.csv')

work = os.path.join(aws_project_dir,'work/')
outdir = os.path.join(aws_project_dir,'results/')

#### create local fps and files for testing. 

In [27]:
references = './references'
assets = './assets'
test_dir = 'fq_subsample'
if not os.path.exists(references):
    os.mkdir(references)
if not os.path.exists(assets):
    os.mkdir(assets)
if not os.path.exists(test_dir):
    os.mkdir(test_dir)

## Test pipeline locally on small subsampled fastq files

#### Run test of pipeline on small subsampled dataset.  
cell 1: download 4 fastq files to local fq_subsample dir.   
cell 2: subsample those 4 files

In [28]:
# Download samplesheet.csv to local fq_subsample directory 
fq_subsample_dir = 'fq_subsample'
if not os.path.exists(fq_subsample_dir):
    os.makedirs(fq_subsample_dir)
full_samplesheet = f'{fq_subsample_dir}/aws_samplesheet.csv'
!aws s3 cp $samplesheet_fp $full_samplesheet

download: s3://mukherjee-lab/dimA_RNA_seq_2025/assets/samplesheet.csv to fq_subsample/aws_samplesheet.csv


In [29]:
rows_to_grab = [0, 1] # add 2 0-based indexing row numbers that you want to grab, 
#check out samplesheet and grab a couple of rows
local_samplesheet_df = pd.read_csv(full_samplesheet).iloc[rows_to_grab,:]
local_samplesheet_df

,sample,fastq_1,fastq_2,strandedness,condition,replicate
0,wt_1_light,s3://mukherjee-lab/dimA_RNA_seq_2025/raw/24121...,s3://mukherjee-lab/dimA_RNA_seq_2025/raw/24121...,reverse,wt_light,1
1,wt_2_light,s3://mukherjee-lab/dimA_RNA_seq_2025/raw/24121...,s3://mukherjee-lab/dimA_RNA_seq_2025/raw/24121...,reverse,wt_light,2


In [11]:


R1_SRC, R2_SRC = list(local_samplesheet_df.loc[rows_to_grab[0], ['fastq_1', 'fastq_2']])
R1_SRC_2, R2_SRC_2 = list(local_samplesheet_df.loc[rows_to_grab[1], ['fastq_1', 'fastq_2']])

inpaths = {}
outpaths = {}
inpaths['pair1']=[]
inpaths['pair2']=[]
outpaths['pair1']=[]
outpaths['pair2']=[]

pathR1='fq_subsample/1_R1.fastq.gz'
pathR1_out = 'fq_subsample/1_R1_subsample.fastq.gz'
inpaths['pair1'].append(pathR1)
outpaths['pair1'].append(pathR1_out)
!aws s3 cp $R1_SRC $pathR1

if isinstance(R2_SRC, str):
    pathR2 = 'fq_subsample/1_R2.fastq.gz'
    pathR2_out = 'fq_subsample/1_R2_subsample.fastq.gz'
    inpaths['pair1'].append(pathR2)
    outpaths['pair1'].append(pathR2_out)
    !aws s3 cp $R2_SRC $pathR2

pathR1='fq_subsample/2_R1.fastq.gz'
pathR1_out = 'fq_subsample/2_R1_subsample.fastq.gz'
inpaths['pair2'].append(pathR1)
outpaths['pair2'].append(pathR1_out)
!aws s3 cp $R1_SRC_2 $pathR1
if isinstance(R2_SRC_2, str):
    pathR2 = 'fq_subsample/2_R2.fastq.gz'
    pathR2_out = 'fq_subsample/2_R2_subsample.fastq.gz'
    inpaths['pair2'].append(pathR2)
    outpaths['pair2'].append(pathR2_out)
    !aws s3 cp $R2_SRC_2 $pathR2

download: s3://mukherjee-lab/dimA_RNA_seq_2025/raw/241213-Sampriti-Lab-FR1705-1_R1.fastq.gz to fq_subsample/1_R1.fastq.gz
download: s3://mukherjee-lab/dimA_RNA_seq_2025/raw/241213-Sampriti-Lab-FR1705-1_R2.fastq.gz to fq_subsample/1_R2.fastq.gz
download: s3://mukherjee-lab/dimA_RNA_seq_2025/raw/241213-Sampriti-Lab-FR1705-2_R1.fastq.gz to fq_subsample/2_R1.fastq.gz
download: s3://mukherjee-lab/dimA_RNA_seq_2025/raw/241213-Sampriti-Lab-FR1705-2_R2.fastq.gz to fq_subsample/2_R2.fastq.gz


In [ ]:
i=0
subsample_size = 100000
for pair, in_path_lst in inpaths.items():
    outpath_lst  = outpaths[pair]
    in1=in_path_lst[0]
    out1=outpath_lst[0]
    local_samplesheet_df.loc[rows_to_grab[i], 'fastq_1'] = os.path.abspath(out1)
    #for paired end sequencing
    if len(in_path_lst) > 1:
        in2=in_path_lst[1]
        out2=outpath_lst[1]
        local_samplesheet_df.loc[rows_to_grab[i], 'fastq_2'] = os.path.abspath(out2)
        !fq subsample -n $subsample_size --r1-dst $out1 --r2-dst $out2 $in1 $in2
    else:
        !fq subsample -n $subsample_size --r1-dst $out1 $in1 
    i+=1
    

In [30]:
local_samplesheet = './assets/samplesheet.csv'
local_samplesheet_df = local_samplesheet_df.set_index('sample')
local_samplesheet_df.to_csv(local_samplesheet)     

##### run local pipeline

In [14]:
local_annotation_fp = os.path.join('references', annotation_filename)
local_genome_fp = os.path.join('references', genome_filename)
!aws s3 cp $annotation_fp $local_annotation_fp
!aws s3 cp $genome_fp $local_genome_fp

download: s3://mukherjee-lab/dimA_RNA_seq_2025/references/genomic.gff to references/genomic.gff
download: s3://mukherjee-lab/dimA_RNA_seq_2025/references/GCA_000014625.1_ASM1462v1_genomic.fna to references/GCA_000014625.1_ASM1462v1_genomic.fna


In [17]:
work = 'work'
outdir = 'results'
local_samplesheet = os.path.abspath(local_samplesheet)
local_annotation_fp = os.path.abspath(os.path.join(local_annotation_fp))
local_genome_fp = os.path.abspath(os.path.join(local_genome_fp))
fps = [local_samplesheet, local_annotation_fp, local_genome_fp]
for fp in fps:
    try:
        with open(fp, 'r') as f:
            content = f.read()
    except FileNotFoundError:
        print(f"The file {fp} does not exist.")
# !aws s3 cp s3://mukherjee-lab/dimA_RNA_seq_2025/references/genomic.gtf ./references/
# local_annotation_fp = './references/genomic.gtf'
local_annotation_fp

'/Users/jonathanwinkelman/Trestle_long-term-storage/Mukherjee_lab/dima_RNA_seq_2025/rnaseq_prokaryotes/references/genomic.gff'

In [20]:
!nextflow run main.nf -profile docker --input $local_samplesheet --fasta $local_genome_fp --outdir $outdir --gff $local_annotation_fp -work-dir $work #-resume


Nextflow 24.10.4 is available - Please consider updating your version to it
N E X T F L O W  ~  version 23.10.1
Launching `main.nf` [maniac_carson] DSL2 - revision: 174702b01e
WARN: The following invalid input values have been detected:

* --gff: /Users/jonathanwinkelman/Trestle_long-term-storage/Mukherjee_lab/dima_RNA_seq_2025/rnaseq_prokaryotes/references/genomic.gff




------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/prokrnaseq v1.0dev
------------------------------------------------------
Core Nextflow options
  runName        : maniac_carson
  containerEngine: docker
  launchDir      : /Users/jonathanwinkelman/Trestle_long-term-storage/Mukherjee_lab/dima_RNA_seq_2025/rnaseq_prokaryotes
  workDir        : /Users/jonathanwink

### run on AWS batch

In [37]:
work = os.path.join(aws_project_dir,'work/')
# genome_fp = os.path.join(aws_project_dir, 'references/', genome_filename)
outdir = os.path.join(aws_project_dir,'results')
# input = os.path.join(aws_project_dir,'assets/samplesheet.csv')
# gff = os.path.join(aws_project_dir,'references/sequence.gff3')

In [38]:
annotation_fp

's3://mukherjee-lab/dimA_RNA_seq_2025/references/genomic.gff'

In [39]:
!nextflow run main.nf -profile jonathan --input $samplesheet_fp --fasta $genome_fp --outdir $outdir --gff $annotation_fp -work-dir $work -resume


Nextflow 24.10.4 is available - Please consider updating your version to it
N E X T F L O W  ~  version 23.10.1
Launching `main.nf` [boring_lagrange] DSL2 - revision: 174702b01e
WARN: The following invalid input values have been detected:

* --gff: s3://mukherjee-lab/dimA_RNA_seq_2025/references/genomic.gff




------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/prokrnaseq v1.0dev
------------------------------------------------------
Core Nextflow options
  runName        : boring_lagrange
  containerEngine: docker
  launchDir      : /Users/jonathanwinkelman/Trestle_long-term-storage/Mukherjee_lab/dima_RNA_seq_2025/rnaseq_prokaryotes
  workDir        : /mukherjee-lab/dimA_RNA_seq_2025/work
  projectDir     : /Users/jonathanwinkelma

In [31]:
multiqc = os.path.join(outdir,'multiqc/multiqc_report.html')
!aws s3 cp $multiqc /Users/jonathanwinkelman/Downloads/

download: s3://mukherjee-lab/dimA_RNA_seq_2025/results/multiqc/multiqc_report.html to ../../../../Downloads/multiqc_report.html


In [41]:
for file in os.listdir('./results/bowtie2/'):
    if file.endswith('.bam'):
        print(file)
        fp = f'./results/bowtie2/{file}'
        !samtools index $fp

wt_1_light_T1.bam
wt_3_dark_T1.bam
bphP_1_T1.bam
dimA_1_light_T1.bam
bphP_3_T1.bam
dimA_3_dark_T1.bam
wt_1_dark_T1.bam
dimA_2_light_T1.bam
dimA_1_dark_T1.bam
dimA_3_light_T1.bam
dimA_2_dark_T1.bam
wt_2_light_T1.bam
wt_2_dark_T1.bam
wt_3_light_T1.bam
bphP_2_T1.bam
